### Inertial navigation systems and IMU
Inertial navigation is a self-contained navigation technique in which measurements provided by accelerometers and gyroscopes are used to track the position and orientation of an object relative to a known starting point, orientation and velocity.
See [Wikipedia Inertial navigation system](https://en.wikipedia.org/wiki/Inertial_navigation_system).

Inertial navigation is used in a wide range of applications including the navigation of aircraft, tactical and strategic missiles, spacecraft, submarines and ships. Recent advances in the construction of MEMS devices have made it possible to manufacture small and light inertial navigation systems. 

Inertial Navigation System (**INS**) provides position, velocity and heading information. A INS system is that in which a current state is evaluated by the relative increment from the previous known state.
INS is based on measurements obtained from an Inertial Measurement Unit (IMU).

Inertial measurement units (**IMU**s) typically contain three orthogonal rate-gyroscopes and three orthogonal accelerometers, measuring angular velocity and linear acceleration respectively. By processing signals from these devices it is possible to track the position and orientation
of a device.

Nearly all IMUs fall into one of the two categories according to the frame of reference in which the rate-gyroscopes and accelerometers operate:

* **body frame**: the navigation system’s frame of reference  
* **global frame**: the frame of reference in which we are navigating  

* Stable Platform Systems 
    * The inertial sensors are mounted on a platform which is isolated from any external rotational motion. In other words the platform is held in alignment with the global frame.
    
* Strapdown Systems 
    * The inertial sensors are mounted rigidly onto the device, and therefore output quantities measured in the body frame rather than the global frame.
    
In a strapdown IMU, all inertial sensors are rigidly attached to the unit (no mechanical movement).

In a gimballed IMU, the gyros and accelerometers are isolated from vehicle angular movements by means of gimbals.

**Attitude** is defined by rotations along the three axes which are called roll, pitch and azimuth or yaw. In this work, roll is defined as rotations along the y axis, pitch is rotations along the x axis and azimuth (or yaw) is rotation along the z axis.

### MEMS
Micro-machined ElectroMechanical Systems technology offers rugged, low cost, small and lightweight inertial sensors relative to the other available technologies.
See [Wikipedia Microelectromechanical systems](https://en.wikipedia.org/wiki/Microelectromechanical_systems)

### Gyroscope

A gyroscope gives us the rate of change of the angular position over time (angular velocity) with a unit of [**deg./s**].  

$$ \dot \theta = {d\theta \over dt} $$

$$ \int_0^t  \dot \theta(t)  dt  \approx \sum_0^t  \dot \theta(t) T_s $$


### Accelerometer
The Accelerometer sensor is an inertial-frame sensor, this means that when the device is in free fall, the acceleration is 0 m/s2 in the falling direction, and when a device is laying flat on a table, the acceleration in upwards direction will be equal to the Earth gravity, i.e. g ≡ 9.8 m/s2 as it is measuring the force of the table pushing the device upwards.

Accelerometers today have very low cost but very, very high in performance. Part of the reason for this is that they are manufactured in enormous quantities. They're use to trigger airbags in cars. They're used in digital cameras to work out whether the camera is in landscape or portrait mode. They're used in your phone for all sorts of different applications and they're used for stabilization of unmanned area of vehicles from low cost toys like this to much more sophisticated systems.

The linear acceleration is the acceleration without the gravity, called gravity compensation.

As accelerometers report acceleration, you need to integrate to get velocity, and again to get position.

write an expression for the orientation of the phone with respect to the world coordinate in terms of yaw, pitch and roll angles which are successive rotations about the z-axis, the y-axis and then the x-axis

attach a coordinate frame to the phone: the x-axis is to the right and the y-axis is to the top of the phone and the z-axis is out of the screen.
acceleration of a device sitting flat on a table should be 1G in the positive upward z direction.

problem of determining the orientation of the phone with respect to a world coordinate frame. The world coordinate frame is shown here in blue and its orientated so that the z-axis is upwards and parallel to the gravity vector. Frame B, the body frame is attached rigidly to the phone

Two very simple equations based on the measured acceleration give me the roll and pitch angle of the phone with respect to the world coordinate frame. Then you note in this equation that yaw angle does not appear. A simple way to think about why this is the case, is that the gravity vector has got three elements but its length is fixed therefore we can define it by only two pieces of information. The third element of the gravity vector is redundant. In this case, the two pieces of information that we need are the roll angle and the pitch angle. We do not need the yaw angle. 


To measure the yaw angle, we need to use a **compass**.


 
 
 Register for TYPE_GRAVITY and TYPE_MAGNETIC_FIELD (needed for flat) and do the calculation below.

getRotationMatrix

a[0] a[1] a[2]
a[3] a[4] a[5]
a[6] a[7] a[8]


Calculate pitch = Math.asin(-R[7])
 flat here means the pitch is less than 25 or more than 155 degrees and not flat otherwise.



Using pitch to determine the flatness of the device (I think you can use less than 45 and more than 135 to determine flatness, you just have to experiment to see if the azymuth value is correct)
if the device is not flat the rotation is Math.atan2(a[6], a[7])

If the **device is flat** to start with get the initial azymuth = Math.atan2(R[1], R[4]) and then take the difference with subsequent azymuths to get the rotation.

If the device transition from flat to not flat there should be no problem the rotation is just the rotation for not flat above.

If the device transition from not flat to flat then you need to get the initial azymuth. This initial azymuth can be set to the back camera direction Math.atan2(-a[2], -a[5]), if the device is upright and there is only rotation about the z-axis then this direction should not change. In reallity it will probably change by a few degrees so you will need to take the average.

### Rotation matrix

See [Wikipedia Rotation_matrix](https://en.wikipedia.org/wiki/Rotation_matrix).

In the context of Android sensors, the rotation matrix is telling you how to map a point from the co-ordinate system of the phone (where the phone itself lies in the x-y plane) to the real world North/East/"Gravity-direction" co-ordinate system.

uses a kind of TRIAD algorithms; Rotation matrix (R[]) is composed of gravity, geomagnetic X gravity, gravity X (geomagnetic X gravity) --- X is cross product. 

 azimuth = (float)Math.atan2(R[1], R[4]);
 pitch = (float)Math.asin(-R[7]);
 roll = (float)Math.atan2(-R[6], R[8]);

## Complementary filter
A complementary filter merges fast rotations from the gyroscope with the slower trends from the accelerometer.

In [ ]:
# gy521.py

import time
import paho.mqtt.client as paho
import matplotlib.pyplot as plt

brokerAddr="localhost"
duration  = 15
x         = []
y         = []
z         = []
r         = []
counter   = 0 
maxnum    = 20
endOfJob  = False
dt        = 0
startTime = 0
angle     = 0.0

def on_message(client, userdata, message) :   #define callback
	global counter, maxnum, endOfJob, x, y, z, r, startTime, angle
	now     = time.time() 
	elapsed = now-startTime
	#print( "now=" , now, "elapsed=",  elapsed)
	startTime = now
	counter = counter + 1
	#msg(androidSensor,event,android,none,androidSensor(TYPE,X,Y,Z),MSGNUM)
	evMsg = str( message.payload.decode("utf-8")  )
	#print("evMsg=", evMsg )
	msgitems = evMsg.split(",")
	msgType  = msgitems[4]                  #TYPE
	if msgType == "gyro"    
	x.append( float( msgitems[5] ) )        #X
	y.append( float( msgitems[6] ) )        #Y
	vz = float( msgitems[7].split(')')[0] ) #Z
	z.append( vz )
	
	da    = abs(vz) * elapsed
	angle = angle + da
	r.append( angle )

	print("vz=", ("%.3f" % vz), "angle=", ("%.3f" % angle), "counter=", counter)
	#print("elapsed=", ("%.2f" % elapsed), "da=", ("%.2f" % da) ) 	
	if counter >= maxnum :
		plt.plot(x, color='red')
		plt.plot(y, color='green')
		plt.plot(z, color='blue')
		plt.plot(r, color='black')
		plt.show()
		client.disconnect()

        
        
        
plt.style.use("classic")
client= paho.Client("receiver")      
client.on_message=on_message            # Bind function to callback
client.connect(brokerAddr)              #connect
print("connected to broker ", brokerAddr)
print("subscribing to unibo/qak/events")
client.subscribe("unibo/qak/events")      #subscribe
startTime     = time.time() 
#print( "startTime=" , ti
print( "startTime=" , startTime )
client.loop_start()             #start loop to process received messages
time.sleep(duration)
client.disconnect()             #disconnect
print("bye")
client.loop_stop()              #stop loop    

 [//]: # (TRICK: This syntax works like a comment, and won't appear in any output.)


ref1= [Duck Duck Go](https://duckduckgo.com)  

inline formula $e^{i\pi} + 1 = 0$ bye

$$e^{i\pi} + 1 = 0$$

$$\sqrt{b^2 - 4ac}$$

$$ sum_{k=1}^N k^2 $$


***
![raspgyro1.png](./imgs/raspgyro1.png)
***